In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from ultralytics import YOLO
from tqdm import tqdm  # nice progress bars
import time
import matplotlib.pyplot as plt
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.models import resnet50, ResNet50_Weights

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import seaborn as sns

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

detector = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)
detector.train(
    data='dataset/data.yaml', 
    epochs=100, 
    imgsz=640, 
    batch=16, 
    name='yolo_transfer_model', 
    device=0, 
    patience=50, 
)

Using device: cuda
Ultralytics 8.3.228 🚀 Python-3.13.9 torch-2.9.1+cu128 CUDA:0 (NVIDIA RTX 3500 Ada Generation Laptop GPU, 11874MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_transfer_model3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=Tr

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x760ed24b14e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,  

In [3]:
def evaluate(model, dataloader, class_names, device):
    all_preds = []
    all_labels = []


    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1).cpu()


            all_preds.extend(preds)
            all_labels.extend(labels)


    print("\n===== Classification Report =====\n")
    print(classification_report(all_labels, all_preds, target_names=class_names))


    cm = confusion_matrix(all_labels, all_preds)


    plt.figure(figsize=(7, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.tight_layout()
    plt.savefig("confusion_matrix.png")
    print("Confusion matrix saved as confusion_matrix.png")


    per_class_accuracy = cm.diagonal() / cm.sum(axis=1)
    print("\nPer-class accuracy:")
    for cls, acc in zip(class_names, per_class_accuracy):
        print(f"{cls}: {acc:.4f}")



In [4]:
# RAW Training the Resnet Classifier Model



temp_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder("cropped_dataset/train", transform=temp_transform)
loader = DataLoader(dataset, batch_size=64, shuffle=False)

mean = 0.
std = 0.
total_images = 0

for images, _ in loader:
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    total_images += batch_samples

mean /= total_images
std /= total_images

print("MEAN:", mean)
print("STD:", std)


# Data transformations (augmentation + normalization)
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean.tolist(), std=std.tolist())
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean.tolist(), std=std.tolist())
])

# Load datasets
train_dataset = datasets.ImageFolder('cropped_dataset/train', transform=train_transforms)
val_dataset = datasets.ImageFolder('cropped_dataset/valid', transform=val_transforms)
test_dataset = datasets.ImageFolder('cropped_dataset/test', transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)



MEAN: tensor([0.4917, 0.4541, 0.3436])
STD: tensor([0.1954, 0.1931, 0.1816])


In [5]:
# Model Training
import csv

logfile = "training_log.csv"

# Create and write header
with open(logfile, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["epoch", "train_loss", "val_loss", "val_acc"])


resnet_model = models.resnet50(pretrained=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet_model.to(device)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 100

for epoch in range(num_epochs):
    resnet_model.train()
    train_loss = 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        imgs, labels = imgs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = resnet_model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimiser.step()

        train_loss += loss.item()

    # Validation
    resnet_model.eval()
    val_loss = 0
    val_correct = 0
    total = 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = resnet_model(imgs)

            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = outputs.max(1)
            val_correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss/len(train_loader):.4f}")
    print(f"Val Loss:   {val_loss/len(val_loader):.4f}")
    print(f"Val Acc:    {100*val_correct/total:.2f}%\n")

    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss   = val_loss / len(val_loader)
    val_acc        = 100 * val_correct / total

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {avg_train_loss:.4f}")
    print(f"Val Loss:   {avg_val_loss:.4f}")
    print(f"Val Acc:    {val_acc:.2f}%\n")

    with open(logfile, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([epoch+1, avg_train_loss, avg_val_loss, val_acc])


    torch.save(model.state_dict(), "resnet50_custom.pth")


/home/cormac/anaconda3/envs/CV_sandbox/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/cormac/anaconda3/envs/CV_sandbox/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Epoch 1/100: 100%|██████████| 239/239 [02:02<00:00,  1.94it/s]


Epoch 1/100
Train Loss: 2.3599
Val Loss:   2.1975
Val Acc:    27.74%

Epoch 1/100
Train Loss: 2.3599
Val Loss:   2.1975
Val Acc:    27.74%



Epoch 2/100: 100%|██████████| 239/239 [02:03<00:00,  1.94it/s]


Epoch 2/100
Train Loss: 1.9012
Val Loss:   1.8977
Val Acc:    39.08%

Epoch 2/100
Train Loss: 1.9012
Val Loss:   1.8977
Val Acc:    39.08%



Epoch 3/100: 100%|██████████| 239/239 [02:02<00:00,  1.94it/s]


...

Epoch 98/100
Train Loss: 0.0331
Val Loss:   2.1212
Val Acc:    66.89%

Epoch 98/100
Train Loss: 0.0331
Val Loss:   2.1212
Val Acc:    66.89%



Epoch 99/100: 100%|██████████| 239/239 [02:02<00:00,  1.95it/s]


Epoch 99/100
Train Loss: 0.0423
Val Loss:   2.1520
Val Acc:    67.64%

Epoch 99/100
Train Loss: 0.0423
Val Loss:   2.1520
Val Acc:    67.64%



Epoch 100/100: 100%|██████████| 239/239 [02:02<00:00,  1.95it/s]


Epoch 100/100
Train Loss: 0.0447
Val Loss:   2.0915
Val Acc:    67.64%

Epoch 100/100
Train Loss: 0.0447
Val Loss:   2.0915
Val Acc:    67.64%



In [6]:

evaluate(model, test_loader, dataset.classes, device)


===== Classification Report =====

              precision    recall  f1-score   support

      class0       0.66      0.72      0.69        87
      class1       0.62      0.82      0.71        44
     class10       0.95      0.79      0.86        47
     class11       0.88      0.84      0.86        58
      class2       0.32      0.50      0.39        44
      class3       0.72      0.63      0.67        93
      class4       0.55      0.78      0.65        40
      class5       0.65      0.41      0.50        73
      class6       0.57      0.53      0.55        55
      class7       0.90      0.74      0.81        47
      class8       0.57      0.49      0.53        51
      class9       0.74      0.80      0.77        50

    accuracy                           0.66       689
   macro avg       0.68      0.67      0.67       689
weighted avg       0.68      0.66      0.66       689

Confusion matrix saved as confusion_matrix.png

Per-class accuracy:
class0: 0.7241
class1: 0.8182